In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Generate timestamps (100 rows, 5-minute intervals)
start_time = datetime.now()
timestamps = [start_time + timedelta(minutes=5 * i) for i in range(100)]

# Generate random values
values = np.random.randn(100)  # Random normal values

# Create DataFrame
df = pd.DataFrame({'timestamp': timestamps, 'value': values})

# Display first few rows
print(df.head())

# Save to CSV (optional)
df.to_csv("time_series_data.csv", index=False)


                   timestamp     value
0 2025-01-29 14:45:15.347286 -1.769867
1 2025-01-29 14:50:15.347286 -0.083507
2 2025-01-29 14:55:15.347286  0.718598
3 2025-01-29 15:00:15.347286 -1.075978
4 2025-01-29 15:05:15.347286 -0.194904


In [2]:
from sklearn.preprocessing import MinMaxScaler

# Normalize the data to [0, 1]
scaler = MinMaxScaler()

# Convert timestamp to sequential index
df["time_index"] = np.arange(len(df))  

# Normalize smooth values (important for GAN training)
scaler = MinMaxScaler()
df["smooth_scaled"] = scaler.fit_transform(df[["value"]])

# Keep only relevant columns
data = df[["time_index", "smooth_scaled"]].values

In [3]:
def create_sequences(data, seq_length=48):
    sequences = []
    for i in range(len(data) - seq_length):
        sequences.append(data[i: i + seq_length])
    return np.array(sequences)

sequence_length = 48  # Adjust as needed
train_data = create_sequences(data, sequence_length)

print(f"Training data shape: {train_data.shape}")  # (num_samples, sequence_length, features)

Training data shape: (52, 48, 2)


In [4]:
from timegan import timegan

## Newtork parameters
parameters = dict()

parameters['module'] = 'gru' 
parameters['hidden_dim'] = 24
parameters['num_layer'] = 3
parameters['iterations'] = 5000
parameters['batch_size'] = 128


# Train TimeGAN
synth_data = timegan(train_data, parameters)  # Generate synthetic data

2025-01-29 14:45:19.362212: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-29 14:45:19.364328: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-29 14:45:20.046194: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-29 14:45:20.153304: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738154720.607207 1523812 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738154721.02

ValueError: Cannot convert '<dtype: 'float32'>' to a shape.